In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# python native csv reader
import csv
import re


array([-11.014874, -10.85401 ], dtype=float32)

In [19]:
def read_up_to_comma(string):
    """
    Read a string up to the first comma.
    """
    for i, char in enumerate(string):
        if char == ',':
            return string[1:i]
    print("No comma found in string:", string)
    return None


In [ ]:
VERBOSE = False

rows = []
with open("data/question_pairs.csv", "r") as f:
    lines = f.readlines()
    header_str = lines[0]
    # ['id,"qid1","qid2","question1","question2","is_duplicate";;;;;;;;']
    id = read_up_to_comma(header_str)
    rest = re.findall(r'(?:,)("".*?)";;', header_str)  # list with one string in it
    headers = [id] + re.findall(r'""(.*?)""(?=,|$)', rest[0])
    rows.append(headers)
    nr_cols = len(headers)
    errors = 0
    for l_idx, line in enumerate(lines[1:], start=1):
        try:
            da_str = line
            id = read_up_to_comma(da_str)
            rest = re.findall(r'(?:,)("".*?)";;', da_str) # list with one string in it
            cols = [id] + re.findall(r'""(.*?)(?<!"{5})""(?:,(?! )|$)', rest[0])
            if len(cols) != nr_cols:
                if VERBOSE:
                    print(f"Line {l_idx} has {len(cols)} columns, expected {nr_cols}")
                    print(f"-------- Line content: {line}")
                    print(f"-------- ID: {id}")
                    print(f"-------- rest: {rest}")
                    print(f"-------- Columns: {cols}")
                    print(f"----------------------------------")
                errors += 1
                continue
            rows.append(cols)
            if l_idx % 50000 == 0:
                print(f"Processed {l_idx} rows.")
        except Exception as e:
            errors += 1
            if VERBOSE:
                print(f"Error processing line {l_idx}: {e}")
                print(f"Line content: {line}")
            continue

# for row in rows[:10]:
#     print(row)
# df = pd.DataFrame(rows[1:], columns=rows[0])
# print(f"DataFrame shape: {df.shape}")
# df.head()
print(f"Total errors encountered: {errors}")
   

Processed 50000 lines.
Processed 100000 lines.
Processed 150000 lines.
Processed 200000 lines.
Processed 250000 lines.
Processed 300000 lines.
Processed 350000 lines.
Processed 400000 lines.
Total errors encountered: 50


In [ ]:
df = pd.DataFrame(rows[1:], columns=rows[0])
print(f"DataFrame shape: {df.shape}")
df.info()

DataFrame shape: (404251, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404251 entries, 0 to 404250
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404251 non-null  object
 1   qid1          404251 non-null  object
 2   qid2          404251 non-null  object
 3   question1     404251 non-null  object
 4   question2     404251 non-null  object
 5   is_duplicate  404251 non-null  object
dtypes: object(6)
memory usage: 18.5+ MB


In [53]:
df["id"] = df["id"].astype(int)
df["qid1"] = df["qid1"].astype(int)
df["qid2"] = df["qid2"].astype(int)
df.loc[df["id"] == 279143, "is_duplicate"] = 0
df["is_duplicate"] = df["is_duplicate"].astype(int)

# for row in df.itertuples():
#     try:
#         dup = int(row.is_duplicate)
#     except ValueError:
#         print(f"ValueError for row {row.id}: {row.is_duplicate}")
        

In [59]:
df = df.set_index("id")

In [85]:
errs = 0
for row in df.itertuples():
    if row.question1.__contains__('";"'):
        #print(f"Row {row.Index} contains a double quote in question1: {row.question1}")
        if not row.question1.__contains__('""""'):
            errs += 1
            print(f"--- Row {row.Index} contains quote but not four double quotes")
            print(f"Row {row.Index} question1: {row.question1}")
            df.loc[row.Index, "question1"] = row.question1.replace('";"', ',')
            print(f"... replacing with comma")
            print(f"Row {row.Index} question1: {df.loc[row.Index, 'question1']}")
        else:
            print(f"Row {row.Index}: {row.question1}")
            df.loc[row.Index, "question1"] = row.question1.replace('""""', '"') 
            print(f"Row {row.Index}: {df.loc[row.Index, 'question1']}")
    if row.question2.__contains__('";"'):
        print(f"Row {row.Index}: {row.question2}")
        df.loc[row.Index, "question2"] = row.question1.replace('";"', ',')
        print(f"Row {row.Index}: {df.loc[row.Index, 'question2']}")
    if row.question2.__contains__('""""'):
        print(f"Row {row.Index}: {row.question2}")
        df.loc[row.Index, "question2"] = row.question2.replace('""""', '"')
        print(f"Row {row.Index}: {df.loc[row.Index, 'question2']}")
                #print(f"Row {row.Index} contains a double quote in question2: {row.question2}")

print(f"Total errors in quotes: {errs}")

Row 8: When do you use """"&"""" instead of """"and""""?
Row 8: When do you use "&" instead of "and"?
Row 80: Why India does not apply the """"Burma-Rohingya model"""" to deport illegal Bangladeshis?
Row 80: Why India does not apply the "Burma-Rohingya model" to deport illegal Bangladeshis?
Row 117: How do Winston and Julia differ in the novel """"1984""""? How are they similar?
Row 117: How do Winston and Julia differ in the novel "1984"? How are they similar?
Row 167: Which is correct: """"speak with"""" or """"speak to""""?
Row 167: Which is correct: "speak with" or "speak to"?
Row 242: In """"A Raisin in the Sun"""", why do Walter and Beneatha argue?
Row 242: In "A Raisin in the Sun", why do Walter and Beneatha argue?
Row 297: Why do people not question """"news"""" posts and opinions posted on social media websites?
Row 297: Why do people not question "news" posts and opinions posted on social media websites?
Row 316: People say """"don't try to please others."""" Does being nice 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 1.2 MB/s eta 0:00:0000:0100:01m


In [90]:
df.to_parquet("data/question_pairs_cleaned.parquet")
df.to_json("data/question_pairs_cleaned.json") 

In [93]:
print("Original df shape:", df.shape)
print("Original df info:")
df.info()
print(df.head())

Original df shape: (404251, 5)
Original df info:
<class 'pandas.core.frame.DataFrame'>
Index: 404251 entries, 0 to 404289
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   qid1          404251 non-null  int64 
 1   qid2          404251 non-null  int64 
 2   question1     404251 non-null  object
 3   question2     404251 non-null  object
 4   is_duplicate  404251 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 34.6+ MB
    qid1  qid2                                          question1  \
id                                                                  
0      1     2  What is the step by step guide to invest in sh...   
1      3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2      5     6  How can I increase the speed of my internet co...   
3      7     8  Why am I mentally very lonely? How can I solve...   
4      9    10  Which one dissolve in water quikly sugar, salt...   

          

In [91]:
df_test_load = pd.read_parquet("data/question_pairs_cleaned.parquet")
print(f"Test load shape: {df_test_load.shape}")
print(f"Test load info:")
df_test_load.info()
print(f"Test load head:")
print(df_test_load.head())

Test load shape: (404251, 5)
Test load info:
<class 'pandas.core.frame.DataFrame'>
Index: 404251 entries, 0 to 404289
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   qid1          404251 non-null  int64 
 1   qid2          404251 non-null  int64 
 2   question1     404251 non-null  object
 3   question2     404251 non-null  object
 4   is_duplicate  404251 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 18.5+ MB
Test load head:
    qid1  qid2                                          question1  \
id                                                                  
0      1     2  What is the step by step guide to invest in sh...   
1      3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   
2      5     6  How can I increase the speed of my internet co...   
3      7     8  Why am I mentally very lonely? How can I solve...   
4      9    10  Which one dissolve in water quikly sugar, salt...   

In [95]:
all(df == df_test_load)

True